https://en.wikipedia.org/wiki/Readers%E2%80%93writers_problem

In [ ]:
import asyncio

r_resource = asyncio.Lock()
r_time = .5


async def reader(index):
    async with r_resource:
        await asyncio.sleep(r_time)
        print(f"reader {index}")
        return index


async def writer(index):
    async with r_resource:
        await asyncio.sleep(r_time)
        print(f"writer {index}")
        return index

reader_tasks = [
    asyncio.create_task(reader(x))
    for x in range(10)
]

writer_tasks = [
    asyncio.create_task(writer(x))
    for x in range(1)
]

await asyncio.gather(*reader_tasks, *writer_tasks)

## First readers–writers problem

In [ ]:
import asyncio

resource = asyncio.Lock()
r_resource = asyncio.Lock()
r_count = 0
r_time = .5
w_time = r_time * 2


async def reader(index):
    global r_count
    # print(f"begin reader {index}")
    # await asyncio.sleep(r_time)
    # print(f"try start reader {index}")
    async with r_resource:
        r_count = r_count + 1
        if r_count == 1:
            # print(f"resource acquire start reader {index}")
            await resource.acquire()
            print(f"resource acquire end reader {index}")

    print(f"start do reader {index}")
    await asyncio.sleep(r_time)
    print(f"end do reader {index}")

    async with r_resource:
        r_count = r_count - 1
        if r_count == 0:
            # print(f"resource release start reader {index}")
            resource.release()
            print(f"resource release end reader {index}")
        print(f"end reader {index}")


async def writer(index):
    # print(f"begin writer {index}")
    # await asyncio.sleep(w_time)
    print(f"try start writer {index}")
    async with resource:
        print(f"start writer {index}")
        await asyncio.sleep(w_time)
        print(f"end writer {index}")
        return index

_ = asyncio.gather(
    *[
        asyncio.create_task(x)
        for x in [
            reader(0),
            reader(1),
            writer(0),
            writer(1),
            reader(2),
            reader(3),
            reader(4),
        ]
    ]
)

## Third readers–writers problem